<a href="https://colab.research.google.com/github/KendallScott/Capstone_alt/blob/main/Data_Cleanup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

References:
https://geemap.org/notebooks/114_dynamic_world/


# Imports and Setup

In [1]:
!pip -q install --upgrade folium
!pip -q install geopandas
!pip -q install geojson
!pip -q install eeconvert

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.7/135.7 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 108.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 10.4 MB/s eta 0:00:00


In [2]:
# reminder that if you are installing libraries in a Google Colab instance you will be prompted to restart your kernal

try:
    import geemap, ee
except ModuleNotFoundError:
    if 'google.colab' in str(get_ipython()):
        print("package not found, installing w/ pip in Google Colab...")
        !pip install geemap
    else:
        print("package not found, installing w/ conda...")
        !conda install mamba -c conda-forge -y
        !mamba install geemap -c conda-forge -y
    import geemap, ee

package not found, installing w/ pip in Google Colab...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 77.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 72.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.5/130.5 kB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 100.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 86.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.5/46.5 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 100.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.7/56.7 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 81.1 MB/s eta 0:00:00
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11124 sha256=ee9e68bd0ad7b4

In [3]:
pip install eemont

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.7/134.7 kB 7.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.7/224.7 kB 24.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for eemont: filename=eemont-0.3.6-py3-none-any.whl size=135743 sha256=869a5ec33efeded2777637680b95980bfc4b6dc59b7c7709fd41b51daee299ea
  Stored in directory: /root/.cache/pip/wheels/c2/79/2c/4fed17c3d3b466bbf4fe5872eec11f189147043b01152a4f75
  Created wheel for ee_extra: filename=ee_extra-0.0.15-py3-none-any.whl size=236757 sha256=ed7aef4bdc63b67f3c903dae84d132154bd263662bf924be88edb32742ea34d9
  Stored in directory: /root/.cache/pip/wheels/29/96/0e/4e36b0dfd85e16867723df739294c0aa45a65b191adac4d959
Successfully built eemont ee_extra


In [4]:
# Standard imports
import requests
import json

import pandas as pd
import numpy as np
from PIL import Image

# Geospatial processing packages
import geopandas as gpd
import geojson

import shapely
from shapely.geometry import box

# Mapping and plotting libraries
import matplotlib.pyplot as plt
import matplotlib.colors as cl
import ee
import eeconvert as eec
import geemap
import folium
import eemont

import geopandas as gpd
from shapely.geometry import shape

### Mount Drive

In [5]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


### Authenticate Google Earth Engine
Make sure you have signed up for access to Google Earth Engine at https://signup.earthengine.google.com/#!/. Once your request has been approved, you should be able to access Google Earth Engine at https://code.earthengine.google.com/.

In [6]:
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=wAgihVCoUzqMsjrb3saG4Dt34hc_ALt8eDDHqHG3CZU&tc=IYzNs6AQm0HLdl6JRmKet1C3DwWF4zo49IKC-BgaiGU&cc=JT-PbDoz0-6yJNsC9ss4HX_AEC9CV1qgmGOoAd_QhMw

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AZEOvhXXsRxA_5tJWlnEYRnqE9E6BqxU5Tn0UgO_QIYofL_QAdsgQbPaztM

Successfully saved authorization token.


In [7]:
# Read the GeoJSON file
mines = gpd.read_file('/content/drive/MyDrive/OSMRE_GEOMINE_GEOJSON.geojson')


# Read data using GeoPandas
print("Data dimensions: {}".format(mines.shape))
mines.sample(3)

Data dimensions: (25124, 31)


,OBJECTID,permittee,company,coalmine_op_status,mine_name,permit_id,msha_id,state_tribe_mine_id,national_id,coal_bed_names,...,mountaintop,steep_slope,highwall,auger,comment,contact,information_link,shape_Length,shape_Area,geometry
25049,25050,Black Panther Coal Company Inc,Black Panther Coal Company Inc,7,Stevenson Mine,P3429,NaN,P3429,ALP3429,NaN,...,NaN,NaN,NaN,NaN,State defined mine status: Closed. May be eith...,8,https://surface-mining.alabama.gov/,3216.592003,219214.810821,"MULTIPOLYGON (((-9556114.820 4158596.424, -955..."
9509,9510,SPICEWOOD HOLLOW MINING,SPICEWOOD HOLLOW MINING,5,NaN,8610220,14129,NaN,KY8610220,HAZARD #4,...,0.0,0.0,0.0,0.0,NaN,3,https://eec.ky.gov/Natural-Resources/Mining/Pa...,803.258777,13093.024951,"MULTIPOLYGON (((-9307891.847 4427017.679, -930..."
19187,19188,REVELATION ENERGY LLC,REVELATION ENERGY LLC,1,NaN,8665166,17128,NaN,KY8665166,"ELKHORN #1, ELKHORN #2, HAZARD #4",...,0.0,0.0,0.0,0.0,NaN,3,https://eec.ky.gov/Natural-Resources/Mining/Pa...,4428.055381,350314.797153,"MULTIPOLYGON (((-9271753.584 4438009.248, -927..."


In [8]:
mines.sample(10)

,OBJECTID,permittee,company,coalmine_op_status,mine_name,permit_id,msha_id,state_tribe_mine_id,national_id,coal_bed_names,...,mountaintop,steep_slope,highwall,auger,comment,contact,information_link,shape_Length,shape_Area,geometry
12879,12880,IBEN INC,IBEN INC,5,NaN,679577X,NaN,NaN,KY679577X,UNKNOWN,...,0.0,0.0,0.0,0.0,NaN,3,https://eec.ky.gov/Natural-Resources/Mining/Pa...,5798.224048,4.426952e+05,"MULTIPOLYGON (((-9630592.791 4468714.344, -963..."
13210,13211,BEAR GRASS COAL COMPANY,BEAR GRASS COAL COMPANY,7,NaN,8130024,NaN,NaN,KY8130024,#4 RIDER,...,0.0,0.0,0.0,0.0,NaN,3,https://eec.ky.gov/Natural-Resources/Mining/Pa...,626.293513,4.458829e+03,"MULTIPOLYGON (((-9271643.768 4496561.239, -927..."
4566,4567,NaN,,4,NaN,0665109,NaN,NaN,KY0665109,,...,0.0,0.0,0.0,0.0,NaN,3,https://eec.ky.gov/Natural-Resources/Mining/Pa...,859.951977,1.195683e+04,"MULTIPOLYGON (((-9268965.863 4453370.171, -926..."
14280,14281,RONALD CHAMBERS,RONALD CHAMBERS,7,NaN,9180191,NaN,NaN,KY9180191,RIVER GEM,...,0.0,0.0,0.0,0.0,NaN,3,https://eec.ky.gov/Natural-Resources/Mining/Pa...,591.141909,9.675228e+03,"MULTIPOLYGON (((-9368967.514 4391935.127, -936..."
9025,9026,GALE COAL COMPANY INC,GALE COAL COMPANY INC,6,NaN,2980927,NaN,NaN,KY2980927,"ALMA, LOWER ELKHORN",...,0.0,0.0,0.0,0.0,NaN,3,https://eec.ky.gov/Natural-Resources/Mining/Pa...,11625.287895,9.347581e+05,"MULTIPOLYGON (((-9137191.705 4504659.094, -913..."
9187,9188,NaN,NaN,4,NaN,301074X,NaN,NaN,KY301074X,NaN,...,NaN,NaN,NaN,NaN,PRE-LAW,3,https://eec.ky.gov/Natural-Resources/Mining/Pa...,27653.240418,2.841270e+05,"MULTIPOLYGON (((-9270530.412 4471556.148, -927..."
15556,15557,TRIPLE B CORPORATION,TRIPLE B CORPORATION,7,NaN,8800019,10516,NaN,KY8800019,"BROAS, LOWER PEACH ORCHARD, LOWER RICHARDSON, ...",...,0.0,0.0,0.0,1.0,NaN,3,https://eec.ky.gov/Natural-Resources/Mining/Pa...,22124.391745,2.278953e+06,"MULTIPOLYGON (((-9187228.558 4565584.818, -918..."
16416,16417,NaN,NaN,4,NaN,626077X,NaN,NaN,KY626077X,NaN,...,NaN,NaN,NaN,NaN,PRE-LAW,3,https://eec.ky.gov/Natural-Resources/Mining/Pa...,6341.794687,1.323170e+05,"MULTIPOLYGON (((-9195115.073 4508931.896, -919..."
17130,17131,WHYMORE COAL COMPANY INC,WHYMORE COAL COMPANY INC,7,NaN,9180382,17560,NaN,KY9180382,"JELLICO, BLUE GEM",...,0.0,0.0,0.0,1.0,NaN,3,https://eec.ky.gov/Natural-Resources/Mining/Pa...,3281.006810,3.693101e+05,"MULTIPOLYGON (((-9364855.580 4418058.390, -936..."
22161,22162,"CORESCO, LLC",ALI CO,1,NA,O006383,NaN,NaN,WVO006383,NaN,...,0.0,0.0,NaN,0.0,NaN,2,https://dep.wv.gov/dmr/Pages/default.aspx,1548.207454,1.017261e+05,"MULTIPOLYGON (((-8901569.803 4822765.898, -890..."


In [12]:
mines = mines.drop(columns=['permittee'])
mines = mines.drop(columns=['mine_name'])
mines = mines.drop(columns=['msha_id'])
mines = mines.drop(columns=['state_tribe_mine_id'])
mines = mines.drop(columns=['mountaintop'])
mines = mines.drop(columns=['steep_slope'])
mines = mines.drop(columns=['highwall'])
mines = mines.drop(columns=['auger'])
mines = mines.drop(columns=['comment'])
mines = mines.drop(columns=['coal_bed_names'])
mines = mines.drop(columns=['date_of_map'])
mines = mines.drop(columns=['inspectable_unit_status'])
mines = mines.drop(columns=['permit_approval_date'])
mines = mines.drop(columns=['edit_date'])
mines = mines.drop(columns=['area_mine'])
mines = mines.drop(columns=['contour'])
mines = mines.drop(columns=['permit_application_type'])
mines = mines.drop(columns=['reported_area'])
mines = mines.drop(columns=['permit_application_date'])

mines = mines.rename(columns={'OBJECTID': 'shapeName'})

In [ ]:
mines

In [14]:
# Get the shape geometry for Tennesssee mines
region = eec.gdfToFc(mines)

In [15]:
df = pd.DataFrame()

In [16]:
mines_exploded=mines.explode()
mines_exploded

<ipython-input-16-9f8c0177ef78>:1: FutureWarning: Currently, index_parts defaults to True, but in the future, it will default to False to be consistent with Pandas. Use `index_parts=True` to keep the current behavior and True/False to silence the warning.
  mines_exploded=mines.explode()


shapeName                      company  coalmine_op_status permit_id  \
0     0          1          HIGGINS COAL CO INC                   7      2404   
1     0          2             MIDNIGHT COAL CO                   7      2405   
2     0          3   META ELKHORN MINING CO INC                   7      2756   
3     0          4  JORDAN BROTHERS COAL CO INC                   7      2497   
      1          4  JORDAN BROTHERS COAL CO INC                   7      2497   
...            ...                          ...                 ...       ...   
25119 0      25120  Justice Coal Of Alabama LLC                   1     P3946   
25120 0      25121  Justice Coal Of Alabama LLC                   1     P3946   
25121 0      25122  Justice Coal Of Alabama LLC                   1     P3946   
25122 0      25123  Justice Coal Of Alabama LLC                   1     P3946   
25123 0      25124  Justice Coal Of Alabama LLC                   1     P3946   

        national_id  post_smcra  calculated_area  contact  \
0     0      TN2404         1.0         2.220938        4   
1     0      TN2405         1.0         2.752046        4   
2     0      TN2756         1.0       270.299198        4   
3     0      TN2497         1.0         3.507990        4   
      1      TN2497         1.0         3.507990        4   
...             ...         ...              ...      ...   
25119 0     ALP3946         1.0       154.872159        8   
25120 0     ALP3946         1.0         2.410359        8   
25121 0     ALP3946         1.0         2.430763        8   
25122 0     ALP3946         1.0       116.325330        8   
25123 0     ALP3946         1.0        66.036976        8   

                            information_link  shape_Length    shape_Area  \
0     0               https://www.osmre.gov/   1220.769913  1.354382e+04   
1     0               https://www.osmre.gov/    860.404674  1.678249e+04   
2     0               https://www.osmre.gov/   7219.650616  1.689153e+06   
3     0               https://www.osmre.gov/    688.042643  2.188255e+04   
      1               https://www.osmre.gov/    688.042643  2.188255e+04   
...                                      ...           ...           ...   
25119 0  https://surface-mining.alabama.gov/   5370.984081  9.330482e+05   
25120 0  https://surface-mining.alabama.gov/    534.608228  1.452293e+04   
25121 0  https://surface-mining.alabama.gov/    507.253927  1.464629e+04   
25122 0  https://surface-mining.alabama.gov/   4223.617167  7.008642e+05   
25123 0  https://surface-mining.alabama.gov/   3278.442601  3.979045e+05   

                                                  geometry  
0     0  POLYGON ((-9515951.177 4212750.257, -9515928.8...  
1     0  POLYGON ((-9516024.476 4212708.540, -9516019.2...  
2     0  POLYGON ((-9459511.615 4347543.957, -9459509.4...  
3     0  POLYGON ((-9372128.011 4337061.595, -9372159.8...  
      1  POLYGON ((-9372169.449 4337098.162, -9372176.4...  
...                                                    ...  
25119 0  POLYGON ((-9540804.196 4143151.210, -9540794.6...  
25120 0  POLYGON ((-9541424.447 4142622.814, -9541448.6...  
25121 0  POLYGON ((-9540236.664 4142842.258, -9540258.9...  
25122 0  POLYGON ((-9540725.233 4143212.597, -9540601.6...  
25123 0  POLYGON ((-9539594.620 4143493.033, -9539552.9...  

[54301 rows x 12 columns]

In [18]:
i = 0
polygon = mines_exploded['geometry'].values[i]

In [19]:
# Initial date of interest (inclusive).
i_date = '2021-01-01'
# Final date of interest (exclusive).
f_date = '2021-01-15'

dc = ee.ImageCollection('GOOGLE/DYNAMICWORLD/V1').filterBounds(region).filterDate(i_date, f_date)


In [ ]:
coordinates.bounds()

AttributeError: ignored

In [22]:
pip install topojson

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.4/82.4 kB 9.1 MB/s eta 0:00:00


In [23]:
import topojson as tp
topo = tp.Topology(mines_exploded, prequantize=False)
mines_exploded = topo.toposimplify(2).to_gdf()

In [ ]:
i=1

# Initial date of interest (inclusive).
i_date = '2021-01-01'
# Final date of interest (exclusive).
f_date = '2021-01-15'

# Extract the coordinates from the polygon
polygon = mines_exploded['geometry'].values[i]

#needs adjusting for different geometry types
coordinates = polygon.exterior.coords[:]
projected_polygon_shape = np.asarray(coordinates)

# Create an ee.Geometry object from the coordinates
polygon = ee.Geometry.MultiPolygon(coordinates)

# Get dynamic world image collection by asset ID
dc = ee.ImageCollection('GOOGLE/DYNAMICWORLD/V1').filterDate(i_date, f_date)

# get band names, equal to column namesbandnames = dc.first().bandNames().getInfo()[:-1]

# Set the maxError parameter
max_error = 1  # Specify your desired maxError value

# Project the geometry coordinates
projected_polygon = polygon.transform('EPSG:4326', max_error)

#decrease scale to get more rows of data/granularity
ac = dc.getRegion(projected_polygon, scale=50000000000 ).getInfo()
df_constructor =pd.DataFrame(ac)

df_constructor.columns = df_constructor.iloc[0]
df_constructor = df_constructor.tail(-1)
df_constructor['company'] = mines_exploded['company'].values[i]
df_constructor['shapeName'] = mines_exploded['shapeName'].values[i]
df_constructor['start_date'] = i_date
df_constructor['end_Date'] = f_date
df = df.append(df_constructor)


In [ ]:
i=0

# Initial date of interest (inclusive).
i_date = '2021-01-01'
# Final date of interest (exclusive).
f_date = '2021-01-15'

df_contructor = []
while i<=len(mines_exploded):
 try:
    # Extract the coordinates from the polygon
    polygon = gdf_simplified['geometry'].values[i]

    #needs adjusting for different geometry types
    coordinates = polygon.exterior.coords[:]
    projected_polygon_shape = np.asarray(coordinates)

    projected_polygon_shape = projected_polygon_shape[z]
    bound_object = ee.Geometry.MultiPolygon(coordinates)
    bound_object = bound_object.bounds()

    # Create an ee.Geometry object from the coordinates
    polygon = ee.Geometry.MultiPolygon(coordinates)

    # Get dynamic world image collection by asset ID
    dc = ee.ImageCollection('GOOGLE/DYNAMICWORLD/V1').filterDate(i_date, f_date)

    # get band names, equal to column namesbandnames = dc.first().bandNames().getInfo()[:-1]

    # Set the maxError parameter
    max_error = 1  # Specify your desired maxError value

    # Project the geometry coordinates
    projected_polygon = polygon.transform('EPSG:4326', max_error)

    #decrease scale to get more rows of data/granularity
    ac = dc.getRegion(projected_polygon, scale=50000000000 ).getInfo()
    df_constructor =pd.DataFrame(ac)

    df_constructor.columns = df_constructor.iloc[0]
    df_constructor = df_constructor.tail(-1)
    df_constructor['company'] = mines_exploded['company'].values[i]
    df_constructor['shapeName'] = mines_exploded['shapeName'].values[i]
    df_constructor['start_date'] = i_date
    df_constructor['end_Date'] = f_date
    df = df.append(df_constructor)


        print(mines_exploded['company'].values[i])
        i+=1
 except:
          i+=1
          pass



IndentationError: ignored

In [ ]:
df['company'].unique()

In [ ]:
df

In [ ]:
len(df['company'].unique())

In [ ]:
len(mines_exploded['company'].unique())